In [20]:
import pandas as pd 

df_domain = pd.read_csv('domain_page_rank_2017.csv')

# count the number of nan in the page_rank column
df_domain.page_rank.isnull().sum()  

44254

In [22]:
df_domain.head()

,hostnames,country,alpha2,alpha3,hostnames_reverse,domain,page_rank
0,www.moh.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.moh.www,ae.gov.moh,11450.0
1,services.moid.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.moid.services,ae.gov.moid.services,20930590.0
2,moh.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.moh,ae.gov.moh,11450.0
3,webmail.moi.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.moi.webmail,ae.gov.moi.webmail,21177815.0
4,academy.adjd.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.adjd.academy,ae.gov.adjd.academy,NaN


In [23]:
df_host = pd.read_csv('host_name_page_rank_2017.csv')
df_host.page_rank.isnull().sum()

75212

In [24]:
df_host.head()

,hostnames,country,alpha2,alpha3,hostnames_reverse,domain,page_rank
0,www.filmdubai.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.filmdubai.www,ae.gov.filmdubai,NaN
1,www.am.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.am.www,ae.gov.am,NaN
2,mocd.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.mocd,ae.gov.mocd,67438003.0
3,careers.drydocks.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.drydocks.careers,ae.gov.drydocks.careers,19974452.0
4,summer.youth.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.youth.summer,ae.gov.youth.summer,NaN


In [25]:
# how to merge two dataframes, fill out the missing values in the page_rank column
df_domain = df_domain.merge(df_host, on='hostnames', how='left')
# we fill out the missing values in the page_rank_x column with the values in the page_rank_y column
df_domain.page_rank_x.fillna(df_domain.page_rank_y, inplace=True)
df_domain.drop('page_rank_y', axis=1, inplace=True)
df_domain.rename(columns={'page_rank_x':'page_rank'}, inplace=True)
df_domain.page_rank.isnull().sum()

43855

In [26]:
# drop the columns that end with _y
df_domain.drop(df_domain.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
# rename the columns that end with _x
df_domain.rename(columns=lambda x: x.replace('_x', ''), inplace=True)
df_domain.head()


,hostnames,country,alpha2,alpha3,hostnames_reverse,domain,page_rank
0,www.moh.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.moh.www,ae.gov.moh,11450.0
1,services.moid.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.moid.services,ae.gov.moid.services,20930590.0
2,moh.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.moh,ae.gov.moh,11450.0
3,webmail.moi.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.moi.webmail,ae.gov.moi.webmail,21177815.0
4,academy.adjd.gov.ae,United Arab Emirates (UAE),AE,ARE,ae.gov.adjd.academy,ae.gov.adjd.academy,NaN


In [31]:
# write the file to csv
df_domain.to_csv('government_websites_page_rank_2017.csv', index=False)

In [27]:
# group by country and select the top 500 based on page_rank

df_group_country = df_domain.groupby('country').apply(lambda x: x.nsmallest(500, 'page_rank')).reset_index(drop=True)
df_group_country.head()

,hostnames,country,alpha2,alpha3,hostnames_reverse,domain,page_rank
0,budget.gouv.cd,"""Congo","Democratic Republic of the""",CD,COD,cd.gouv.budget,12237363.0
1,medd.gouv.cd,"""Congo","Democratic Republic of the""",CD,COD,cd.gouv.medd,16589174.0
2,www.senat.cd,"""Congo","Democratic Republic of the""",CD,COD,cd.senat.www,NaN
3,www.investindrc.cd,"""Congo","Democratic Republic of the""",CD,COD,cd.investindrc.www,NaN
4,www.budget.gouv.cd,"""Congo","Democratic Republic of the""",CD,COD,cd.gouv.budget.www,NaN


In [28]:
# count the numer of domains per country
df_group_country.country.value_counts()

Tanzania               500
Kazakhstan             500
Peru                   500
Pakistan               500
Nepal                  500
                      ... 
Country                  1
"sk.gov.vlada            1
dps.staging.nv.gov"      1
nwsb.go.ke"              1
perpusnas.go.id"         1
Name: country, Length: 209, dtype: int64

In [30]:
# filter the country that has great than 500 domains

df_group_country.country.value_counts().reindex().to_csv('country_count.csv')